In [2]:
""" Convert GeoPackage to Bigquery Compatible Format

Approach:

read geospatial datatype

ogr2ogr to converto to csv file

TODO:

allow direct upload to GCS instead of writing to VM.  
Add other geospatial datastypes.
Add multiprocessing (or Apache Beam if I want to make it fancy)

"""

SCRIPT_NAME = "Y2018M09D17_RH_GADM_Ogr2Ogr_V01"
OUTPUT_VERSION = 1

GCS_OUTPUT_PATH = "gs://aqueduct30_v01/{}".format(SCRIPT_NAME)

ec2_input_path = "/volumes/data/%s/input_V%0.2d" %(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/%s/output_V%0.2d" %(SCRIPT_NAME,OUTPUT_VERSION)


In [3]:
!rm -r {ec2_input_path}
!mkdir -p {ec2_input_path}
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/input_V01': No such file or directory
rm: cannot remove '/volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/output_V01': No such file or directory


In [4]:
filename_zip = "gadm36_gpkg.zip" 
filename_gpkg = "gadm36.gpkg"
url = "https://biogeo.ucdavis.edu/data/gadm3.6/%s" %filename_zip
file_path_zip = "%s/%s" %(ec2_input_path, filename_zip)
file_path_gpkg = "%s/%s" %(ec2_input_path, filename_gpkg)

In [5]:
!wget {url} -P {ec2_input_path}

--2018-09-17 06:00:15--  https://biogeo.ucdavis.edu/data/gadm3.6/gadm36_gpkg.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/gadm3.6/gadm36_gpkg.zip [following]
--2018-09-17 06:00:16--  https://data.biogeo.ucdavis.edu/data/gadm3.6/gadm36_gpkg.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 574403668 (548M) [application/zip]
Saving to: '/volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/input_V01/gadm36_gpkg.zip'

gadm36_gpkg.zip     100%[===================>] 547.79M  9.88MB/s    in 56s     

2018-09-17 06:01:14 (9.70 MB/s) - '/volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/input_V01/gadm36_gpkg.zip'

In [6]:
!unzip {file_path_zip} -d {ec2_input_path}

Archive:  /volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/input_V01/gadm36_gpkg.zip
  inflating: /volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/input_V01/gadm36.gpkg  


In [7]:
ec2_output_file_path = "{}/gadm36.csv".format(ec2_output_path)

In [14]:
command = "/opt/anaconda3/envs/python35/bin/ogr2ogr -f 'CSV' -lco GEOMETRY=AS_WKT {} {}/{}".format(ec2_output_file_path,ec2_input_path,filename_gpkg)

In [15]:
command

"/opt/anaconda3/envs/python35/bin/ogr2ogr -f 'CSV' -lco GEOMETRY=AS_WKT /volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/output_V01/gadm36.csv /volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/input_V01/gadm36.gpkg"

In [16]:
import subprocess

In [13]:
response = subprocess.check_output(command,shell=True)

CalledProcessError: Command 'ogr2ogr -f 'CSV' -lco GEOMETRY=AS_WKT /volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/output_V01/gadm36.csv /volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/input_V01/gadm36.gpkg' returned non-zero exit status 127

In [17]:
!gsutil -m cp -r {ec2_output_path} {GCS_OUTPUT_PATH}

Copying file:///volumes/data/Y2018M09D17_RH_GADM_Ogr2Ogr_V01/output_V01/gadm36.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][  3.4 GiB/  3.4 GiB] 100% Done  52.1 MiB/s ETA 00:00:00           
Operation completed over 1 objects/3.4 GiB.                                      
